In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


ModuleNotFoundError: No module named 'keras'

In [ ]:
DATA = "ARTINT"

In [3]:
d_feature_vector = pickle.load(open(DATA+"_train_title_feature_vector.p", "rb"))
d_feature_vector_test = pickle.load(open(DATA+"_test_title_feature_vector.p", "rb"))


In [4]:

x_train = []
y_train = []
for i in d_feature_vector.keys():
    x_train.append(d_feature_vector[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)


(3056, 1875)


In [5]:

x_test = []
y_test = []
for i in d_feature_vector_test.keys():
    x_test.append(d_feature_vector_test[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_test.append(label)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape)


(768, 1875)


In [6]:
c = 0
for i in y_test:
    if i == 0.0:
        c = c+ 1
print(c)

364


In [7]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(50, activation = "relu", input_dim=1875))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                93800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 98,951
Trainable params: 98,951
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile( optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [9]:
results = model.fit( x_train,y_train, epochs= 5, batch_size = 32, validation_data = (x_test, y_test))


Train on 3056 samples, validate on 768 samples
Epoch 1/5
3056/3056 [==============================] - 1s 257us/step - loss: 0.6244 - acc: 0.6394 - val_loss: 0.4550 - val_acc: 0.8294
Epoch 2/5
3056/3056 [==============================] - 0s 88us/step - loss: 0.3879 - acc: 0.8442 - val_loss: 0.3840 - val_acc: 0.8451
Epoch 3/5
3056/3056 [==============================] - 0s 94us/step - loss: 0.2721 - acc: 0.8982 - val_loss: 0.4211 - val_acc: 0.8216
Epoch 4/5
3056/3056 [==============================] - 0s 91us/step - loss: 0.2152 - acc: 0.9166 - val_loss: 0.4510 - val_acc: 0.8177
Epoch 5/5
3056/3056 [==============================] - 0s 140us/step - loss: 0.1788 - acc: 0.9310 - val_loss: 0.4675 - val_acc: 0.8242


In [11]:
predicted =  model.predict(x_test)

In [13]:
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [27]:
print(f1_score(y_test, predicted, average="macro"))
print(precision_score(y_test, predicted, average="macro"))
print(recall_score(y_test, predicted, average="macro"))    

0.8242041456487644
0.8254733954795739
0.825984658905451


In [19]:
x = classification_report(y_test,predicted)

In [24]:
print(x)

              precision    recall  f1-score   support

         0.0       0.79      0.86      0.82       364
         1.0       0.86      0.79      0.83       404

   micro avg       0.82      0.82      0.82       768
   macro avg       0.83      0.83      0.82       768
weighted avg       0.83      0.82      0.82       768



In [28]:
model.save(DATA+'only_title_model')